# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "Humans" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156780 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Fitting models

In [2]:
model0 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“unable to evaluate scaled gradient”
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge: degenerate  Hessian with 2 negative eigenvalues”


In [3]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Participant_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model1

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Participant_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
1289.9295 1337.6539 -636.9648 1273.9295      2872 
Random effects:
 Groups         Name        Std.Dev.
 Stimulus_ID    (Intercept) 1.6691  
 Participant_ID (Intercept) 0.6123  
Number of obs: 2880, groups:  Stimulus_ID, 900; Participant_ID, 64
Fixed Effects:
                       (Intercept)                           Angleleft  
                            7.3629                             -3.1946  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -3.1319                             -1.2947  
scale(n_candidates, scale = FALSE)                              ActorY  
                  

In [4]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [5]:
anova(model1, model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model2,7,1302.104,1343.862,-644.0518,1288.104,NA,NA,NA
model1,8,1289.930,1337.654,-636.9648,1273.930,14.17404,1,0.0001666543


In [6]:
model <- model1

# Interpretation

In [7]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID) + (1 | Participant_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  1289.9   1337.7   -637.0   1273.9     2872 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.4876  0.0368  0.0893  0.1927  1.9914 

Random effects:
 Groups         Name        Variance Std.Dev.
 Stimulus_ID    (Intercept) 2.7860   1.6691  
 Participant_ID (Intercept) 0.3749   0.6123  
Number of obs: 2880, groups:  Stimulus_ID, 900; Participant_ID, 64

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          7.3629     0.5270  13.971  < 2e-16 ***
Angleleft                           -3.1946     0.4221  -7.568 3.78e-14 ***
Angleright                       

In [8]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),7.3628635,0.5270027,13.971206,2.336359e-44,2.539833,2.8989553
Angleleft,-3.1945815,0.4220948,-7.568399,3.778531e-14,2.539833,-1.2577917
Angleright,-3.1318812,0.4237471,-7.390921,1.458154e-13,2.539833,-1.2331050
"scale(Proximity, scale = FALSE)",-1.2947018,0.1682499,-7.695112,1.413702e-14,2.539833,-0.5097586
"scale(n_candidates, scale = FALSE)",-0.6509610,0.1527348,-4.262034,2.025750e-05,2.539833,-0.2563007
ActorY,-0.9935805,0.2397990,-4.143389,3.422100e-05,2.539833,-0.3911991


In [9]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.078483066,0.01354180,-5.7956149,6.807128e-09,27.1303065,-0.10502451,-0.05194162
(right) - (front),-0.071973439,0.01246676,-5.7732285,7.776686e-09,26.9381974,-0.09640783,-0.04753904
(right) - (left),0.006509627,0.01406623,0.4627841,6.435191e-01,0.6359451,-0.02105967,0.03407893


In [10]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-1.294702,0.1682579,-7.694748,1.41773e-14,46.00341,-1.624481,-0.9649225,7.171848,7.171589,7.171718


In [11]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-0.650961,0.1527407,-4.261871,2.027225e-05,15.59013,-0.9503272,-0.3515948,7.171784,7.171653,7.171718


In [12]:
avg_predictions(model, by="Actor", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Y) - (X),-0.04450157,0.0102736,-4.331643,1.480005e-05,16.04404,-0.06463745,-0.02436568
